In [1]:
import numpy as np
import pickle
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, IterableDataset
from players import Model
import torch.optim as optim 
from fastprogress import master_bar, progress_bar
from sklearn.model_selection import train_test_split
import os

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class ChessDataset(IterableDataset):
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        file_itr = np.load(self.filename)
        return zip(file_itr['x'], file_itr['y'])

In [4]:
train_dataset = ChessDataset('data/train/train.npz')
valid_dataset = ChessDataset('data/valid/valid.npz')

train_dataloader = DataLoader(train_dataset, batch_size=1024)
valid_dataloader = DataLoader(valid_dataset, batch_size=1024)

In [5]:
model = Model()
model.to(device)

Model(
  (a1): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (a2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (a3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
  (b1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (c1): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (c2): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (c3): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
  (d1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
  (d2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
  (d3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
  (last): Linear(in_features=128, out_features=1, bias=True)
)

In [14]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
lr_scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, 
    max_lr=3e-4, 
    steps_per_epoch=len(train_dataloader), 
    epochs=10)

In [12]:
epochs = 10
mb = master_bar(range(epochs))
train_losses, val_losses = [], []

for epoch in mb:
    train_running_loss, val_running_loss = 0.0, 0.0
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            dataloader = train_dataloader
        elif phase == 'val':
            model.eval()
            dataloader = val_dataloader
            
        for i, (inputs, results) in enumerate(progress_bar(dataloader, parent=mb)):
            inputs, results = inputs.type(torch.FloatTensor), results.type(torch.FloatTensor)
            inputs, results = inputs.to(device), results.to(device)
            results = results.unsqueeze(-1)

            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, results)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    lr_scheduler.step()
                    train_running_loss += loss.item() * inputs.size(0)
                elif phase == 'val':
                    val_running_loss += loss.item() * inputs.size(0)  
    
    train_loss = train_running_loss/len(x_train)
    val_loss = val_running_loss/len(x_val)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    x = np.arange(0, len(range(epoch+1)), 1)
    graphs = [[x,np.array(train_losses)], [x,np.array(val_losses)]]
    x_bounds = [0, epochs]
    y_bounds = [0,1]
    mb.update_graph(graphs, x_bounds, y_bounds)
    
    print(f"Epoch:{epoch+1} \
            Training Loss:{train_loss} \
            Validation Loss:{val_loss}")

ValueError: too many values to unpack (expected 2)

In [10]:
torch.save(model.state_dict(), 'models/model.pth')